In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
#%matplotlib inline

In [5]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while(1):

    # Take each frame
    _, frame = cap.read()

    # Convert BGR to HSV
    #hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)

    # define range of blue color in HSV
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)

    cv2.imshow('frame',frame)
    cv2.imshow('hsv', hsv)
    cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()

In [3]:
green = np.uint8([[[0,255,0 ]]])
hsv_green = cv2.cvtColor(green,cv2.COLOR_BGR2HSV)
print hsv_green

[[[ 60 255 255]]]


In [3]:
input_image_cv = cv2.imread('train256/16_right.jpeg', 1)
#plt.imshow(input_image_cv)
#plt.show()
cv2.imshow('image',input_image_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()
hls_input_image_cv = cv2.cvtColor(input_image_cv, cv2.COLOR_BGR2HLS)


In [4]:
hls_input_image_cv[10][10]

array([0, 0, 0], dtype=uint8)

In [5]:
def local_contrast_enhancement(input_image, patch_size, color_scale, var_threshold):
    """ local contrast enhancement """
    """ color_scale is the scale of value in each dimension, if [0, 1], input 1; if [0, 255]. input 255 """
    f_min = np.amin(input_image[:, :, 1])
    f_max = np.amax(input_image[:, :, 1])
    res_pic = np.copy(input_image)
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            if (res_pic[i][j][1] == 0):
                continue
            start_pos_i = i - int(patch_size / 2)
            start_pos_j = j - int(patch_size / 2)
            end_pos_i = start_pos_i + patch_size
            end_pos_j = start_pos_j + patch_size
            start_pos_i = max(0, start_pos_i)
            start_pos_j = max(0, start_pos_j)
            end_pos_i = min(input_image.shape[0], end_pos_i)
            end_pos_j = min(input_image.shape[1], end_pos_j)
            patch_mean = input_image[start_pos_i: end_pos_i, start_pos_j : end_pos_j, 1].mean()
            patch_var = input_image[start_pos_i: end_pos_i, start_pos_j : end_pos_j, 1].var()
            if patch_var < var_threshold:
                continue
            res_pic[i][j][1] = color_scale * ((PHI_W(input_image[i][j][1], patch_mean, patch_var) - PHI_W(f_min, patch_mean, patch_var)) / (PHI_W(f_max, patch_mean, patch_var) - PHI_W(f_min, patch_mean, patch_var)))

    return res_pic

In [6]:
def PHI_W(f, patch_mean, patch_var):
    return 1 / (1 + np.exp((patch_mean - f) / (patch_var ** 0.5)))

In [10]:
start = time.time()
output_pic = local_contrast_enhancement(hls_input_image_cv, 25, 255, 30)
print time.time()-start

1.76140785217


In [8]:
res_pic_rbg = cv2.cvtColor(output_pic, cv2.COLOR_HLS2RGB)
plt.imshow(res_pic_rbg)
plt.show()

In [17]:
thresholds = [10, 20, 30, 40, 50, 60]
_, sub_fig = plt.subplots(nrows = 2, ncols = 3)
for i in range(len(thresholds)):
    output_pic = local_contrast_enhancement(hls_input_image_cv, 32, 255, i)
    res_pic_rbg = cv2.cvtColor(output_pic, cv2.COLOR_HLS2RGB)
    sub_fig[i/3][i%3].imshow(res_pic_rbg)
plt.show()

#sub_fig[0].imshow(input_image)
#sub_fig[1].imshow(output_pic)
#plt.show()